In [2]:
import pandas as pd

In [4]:
df=pd.read_csv("train.csv")

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df.shape

(20800, 5)

In [10]:
df.dropna(inplace=True)

In [11]:
#Get all the independent features 
X=df.drop(['label'],axis=1)

In [12]:
y=df['label']

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.15.0'

In [15]:
#Vocabulary size
voc_size=5000

In [16]:
message=X.copy()

In [19]:
#Do stemming and remove stop words
import nltk
import re
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
message.reset_index(inplace=True)

In [28]:

corpus=[]
for i in range(0, len(message)):
    review=re.sub('[^A-Za-z]',' ',message['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [29]:
from tensorflow.keras.preprocessing.text import one_hot

In [31]:
onehot_resp=[one_hot(words,voc_size) for words in corpus]

In [32]:
onehot_resp

[[2698, 4329, 1123, 1149, 26, 4131, 3734, 1445, 2117, 713],
 [1160, 3792, 1023, 4036, 4481, 858, 4777],
 [4683, 4951, 936, 4528],
 [1088, 3190, 1885, 2490, 3183, 4986],
 [4215, 4481, 1331, 1525, 553, 4632, 4481, 2378, 281, 453],
 [4049,
  3788,
  4312,
  1290,
  3500,
  719,
  1724,
  1916,
  4553,
  2127,
  3043,
  1369,
  265,
  4163,
  4777],
 [4605, 3501, 3569, 683, 4155, 1047, 1535, 3070, 799, 835, 2468],
 [4295, 2896, 2362, 2983, 3634, 4117, 719, 2500, 799, 835, 2468],
 [2093, 1595, 1430, 2241, 1083, 2233, 2137, 3911, 719, 3682],
 [3009, 1033, 1897, 390, 3071, 167, 2933, 4033],
 [324, 3164, 4267, 2379, 4806, 357, 1207, 4417, 2598, 4200, 4204],
 [2490, 2667, 26, 2233, 719, 3634],
 [609, 2504, 545, 831, 4422, 4327, 139, 1017, 2376],
 [1263, 1728, 4673, 333, 2486, 6, 2382, 799, 835, 2468],
 [955, 1314, 1867, 4320, 4982, 799, 835, 2468],
 [675, 1260, 2343, 2773, 2400, 4522, 488, 1936, 4351, 3877],
 [4362, 2774, 3792],
 [4989, 1075, 3259, 1312, 719, 3749, 461, 4777],
 [1563, 4126, 102

# Embedding Representation

In [34]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#Do padding before Embedding
sent_len=20
embedded_docs=pad_sequences(onehot_resp,padding='pre',maxlen=sent_len)
print(embedded_docs)

[[   0    0    0 ... 1445 2117  713]
 [   0    0    0 ... 4481  858 4777]
 [   0    0    0 ... 4951  936 4528]
 ...
 [   0    0    0 ...  799  835 2468]
 [   0    0    0 ... 4104 2294 3604]
 [   0    0    0 ... 1362 3324  270]]


In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional

In [36]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [37]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [39]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


192/192 [==============================] - 10s 28ms/step - loss: 0.2983 - accuracy: 0.8618 - val_loss: 0.1964 - val_accuracy: 0.9181
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1336 - accuracy: 0.9491 - val_loss: 0.2052 - val_accuracy: 0.9162
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0874 - accuracy: 0.9686 - val_loss: 0.2705 - val_accuracy: 0.9178
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0595 - accuracy: 0.9789 - val_loss: 0.3049 - val_accuracy: 0.9107
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0379 - accuracy: 0.9873 - val_loss: 0.3849 - val_accuracy: 0.9022
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0267 - accuracy: 0.9916 - val_loss: 0.4451 - val_accuracy: 0.9056
Epoch 7/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0135 - accuracy: 0.9964 - val_loss: 0.5317 - val_accuracy: 0.9

# Performance Metrics And Accuracy

In [40]:
y_pred=model.predict(X_test)

189/189 [==============================] - 3s 8ms/step


In [41]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3132,  287],
       [ 293, 2323]], dtype=int64)

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.903893951946976

In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      3419
           1       0.89      0.89      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

